<a href="https://colab.research.google.com/github/shirish-baral/deep-learning/blob/main/PyTorch_Autograd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
def dy_dx(x):
  return 2*x

In [18]:
dy_dx(3)

6

In [19]:
import math
def dz_dx(x):
  return 2 * x * math.cos(x**2)

In [20]:
dz_dx(2)

-2.6145744834544478

In [21]:
import torch

In [22]:
x = torch.tensor(3.0, requires_grad=True) #True means, we are calculating derivative

In [23]:
y = x**2 # in forward direction

In [24]:
x

tensor(3., requires_grad=True)

In [25]:
y.backward() # to compute derivative or gradient

In [26]:
x.grad # to display value of graidents


tensor(6.)

In [27]:
z = torch.sin(y)
z

tensor(0.4121, grad_fn=<SinBackward0>)

**In forward,**
x -> sqr -> y -> sin -> z

**In backward,**
To compute: dz/dx,

dz/dy -> dy/dx

In [31]:
# Recreate the graph
x = torch.tensor(3.0, requires_grad=True)
y = x**2
z = torch.sin(y)

z.backward()

In [32]:
x.grad

tensor(-5.4668)

In neural network, we have to compute two derivatives:
1. Loss with respect to w
2. Loss with respect to b

dL/dw = dL/dy * dy/dz * dz/dw

dL/db = dL/dy * dy/dz * dz/db

In [35]:
import torch

# Inputs
x = torch.tensor(6.7) # Input feature
y = torch.tensor(0.0) # True label (binary)

w = torch.tensor(1.0) # weights
b = torch.tensor(0.0) # bias


In [36]:
# Binary Cross-Entropy Loss for scalar
def binary_cross_entropy(y_pred, target):
  epsilon = 1e-8 # prevent log()
  y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)
  return -(target * torch.log(y_pred) + (1 - target) * torch.log(1 - y_pred))

In [42]:
from logging import logProcesses
# Forward Pass
z = w * x + b  # weghted sum (linear part)
y_pred = torch.sigmoid(z) # sigmoid activation function

# compute binary cross-entropy loss
loss = binary_cross_entropy(y_pred, y)

In [43]:
loss

tensor(6.7012)

In [40]:
#Derivatives
# 1. dL/d(y_pred) : Loss with respect to the prediction (y_pred)
dloss_dy_pred = (y_pred - y)/(y_pred *(1-y_pred))

# 2. dy_pred /dz: Prediction (y_pred) with respect to z (sigmoid derivative)
dy_pred_dz = y_pred * (1 - y_pred)

# 3. dz/dw and dz/db: z with respect to w and b
dz_dw = x # dz/db = x
dz_db = 1 # dz/db = 1 (bias contributes directly to z)

dL_dw = dloss_dy_pred * dy_pred_dz * dz_dw
dL_db = dloss_dy_pred * dy_pred_dz * dz_db

In [41]:
print(f"Manual Graident of loss w.r.t weight (dw): {dL_dw}")
print(f"Manual Graident of loss w.r.t bias (db): {dL_db}")

Manual Graident of loss w.r.t weight (dw): 6.691762447357178
Manual Graident of loss w.r.t bias (db): 0.998770534992218


# Now, using **Autograd**

In [45]:
x = torch.tensor(6.7)
y = torch.tensor(0.0)

In [46]:
w = torch.tensor(1.0, requires_grad=True)
b = torch.tensor(0.0, requires_grad=True)

In [47]:
w

tensor(1., requires_grad=True)

In [48]:
b

tensor(0., requires_grad=True)

In [50]:
z = w * x+ b
z

tensor(6.7000, grad_fn=<AddBackward0>)

In [51]:
y_pred = torch.sigmoid(z)
y_pred

tensor(0.9988, grad_fn=<SigmoidBackward0>)

In [53]:
loss = binary_cross_entropy(y_pred,y)
loss

tensor(6.7012, grad_fn=<NegBackward0>)

In [54]:
loss.backward()

In [56]:
print(w.grad)
print(b.grad)

tensor(6.6918)
tensor(0.9988)


Loss computed easily with tensors

In [58]:
x = torch.tensor([1.0,2.0,3.0], requires_grad = True)
x

tensor([1., 2., 3.], requires_grad=True)

In [59]:
y = (x**2).mean()
y

tensor(4.6667, grad_fn=<MeanBackward0>)

In [60]:
y.backward()

In [61]:
x.grad

tensor([0.6667, 1.3333, 2.0000])

**Clearing Grad**

When multiple backward path is used, gradients start to accumulate

Graidents are added to previously computed graidents, so we need to clear it

In [81]:
x.grad.zero_() # graidents are cleared after each epoch

tensor(0.)

In [77]:
x = torch.tensor(2.0, requires_grad = True)
x

tensor(2., requires_grad=True)

In [78]:
y = x ** 2
y

tensor(4., grad_fn=<PowBackward0>)

In [79]:
y.backward()

In [80]:
x.grad

tensor(4.)

**Disable Graident Tracking**

Turned off when model is trained and now we need to make predictions. Backward pass is not needed

In [82]:
# option 1 - requires_grad_(False)
# option 2 - detach()
# option 3 - torch.no_grad()

In [83]:
x.requires_grad_(False)

tensor(2.)

In [85]:
y = x ** 2
y

tensor(4.)

In [86]:
# we cannot call y.backward now

In [87]:
x = torch.tensor(2.0, requires_grad = True)
x

tensor(2., requires_grad=True)

In [88]:
z = x.detach()
z

tensor(2.)

In [90]:
y = x ** 2
y

tensor(4., grad_fn=<PowBackward0>)

In [92]:
y1 = z ** 2
y1

tensor(4.)

In [93]:
# we can do y.backward but not y1.backward()

In [95]:
x = torch.tensor(2.0, requires_grad= True)
x

tensor(2., requires_grad=True)

In [96]:
with torch.no_grad():
  y = x ** 2
  y